# Instagram Post Agents
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MervinPraison/PraisonAI/blob/main/cookbooks/notebooks/instagram_post_agents.ipynb)

## Dependencies

In [ ]:
# Install dependencies without output
%pip install praisonai > /dev/null
%pip install praisonai[crewai] > /dev/null
%pip install duckduckgo_search > /dev/null
%pip install python-dotenv > /dev/null

## Tools

In [ ]:
# This program requires addition of a pending "InstagramSearchTool" which will be added soon!
# https://github.com/MervinPraison/PraisonAI-Tools/tree/develop/praisonai_tools/tools
from duckduckgo_search import DDGS
from langchain.tools import tool
from praisonai_tools import BaseTool, ScrapeWebsiteTool

class InternetSearchTool(BaseTool):
    name: str = "InternetSearchTool"
    description: str = "Search Internet for relevant information based on a query or latest news"

    def _run(self, query: str):
        ddgs = DDGS()
        results = ddgs.text(keywords=query, region='wt-wt', safesearch='moderate', max_results=5)
        return results

os.environ["SERPER_API_KEY"] = "ENTER SERPER_API_KEY HERE"
os.environ["BROWSERLESS_API_KEY"] = "ENTER BROWSERLESS_API_KEY HERE"

# import json
# import os

# import requests
# from crewai import Agent, Task
# from langchain.tools import tool
# from unstructured.partition.html import partition_html

# from langchain.llms import Ollama

# class BrowserTools():

#   @tool("Scrape website content")
#   def scrape_and_summarize_website(website):
#     """Useful to scrape and summarize a website content, just pass a string with
#     only the full url, no need for a final slash `/`, eg: https://google.com or https://clearbit.com/about-us"""
#     url = f"https://chrome.browserless.io/content?token={os.environ['BROWSERLESS_API_KEY']}"
#     payload = json.dumps({"url": website})
#     headers = {'cache-control': 'no-cache', 'content-type': 'application/json'}
#     response = requests.request("POST", url, headers=headers, data=payload)
#     elements = partition_html(text=response.text)
#     content = "\n\n".join([str(el) for el in elements])
#     content = [content[i:i + 8000] for i in range(0, len(content), 8000)]
#     summaries = []
#     for chunk in content:
#       agent = Agent(
#           role='Principal Researcher',
#           goal=
#           'Do amazing researches and summaries based on the content you are working with',
#           backstory=
#           "You're a Principal Researcher at a big company and you need to do a research about a given topic.",
#           llm=Ollama(model=os.environ['MODEL']),
#           allow_delegation=False)
#       task = Task(
#           agent=agent,
#           description=
#           f'Analyze and make a LONG summary the content bellow, make sure to include the ALL relevant information in the summary, return only the summary nothing else.\n\nCONTENT\n----------\n{chunk}'
#       )
#       summary = task.execute()
#       summaries.append(summary)
#       content = "\n\n".join(summaries)
#     return f'\nScrapped Content: {content}\n'


# import json
# import os

# import requests
# from langchain.tools import tool


# class SearchTools():

#   @tool("Search internet")
#   def search_internet(query):
#     """Useful to search the internet about a given topic and return relevant
#     results."""
#     return SearchTools.search(query)

#   @tool("Search instagram")
#   def search_instagram(query):
#     """Useful to search for instagram post about a given topic and return relevant
#     results."""
#     query = f"site:instagram.com {query}"
#     return SearchTools.search(query)

#   def search(query, n_results=5):
#     url = "https://google.serper.dev/search"
#     payload = json.dumps({"q": query})
#     headers = {
#         'X-API-KEY': os.environ['SERPER_API_KEY'],
#         'content-type': 'application/json'
#     }
#     response = requests.request("POST", url, headers=headers, data=payload)
#     results = response.json()['organic']
#     stirng = []
#     for result in results[:n_results]:
#       try:
#         stirng.append('\n'.join([
#             f"Title: {result['title']}", f"Link: {result['link']}",
#             f"Snippet: {result['snippet']}", "\n-----------------"
#         ]))
#       except KeyError:
#         next

#     content = '\n'.join(stirng)
#     return f"\nSearch result: {content}\n"



## YAML Prompt

In [ ]:
agent_yaml = """
framework: "crewai"
topic: "Marketing Analysis and Strategy Development"
roles:
  lead_market_analyst:
    role: "Lead Market Analyst"
    backstory: |
      As the Lead Market Analyst at a premier digital marketing firm, you specialize in dissecting online business landscapes.
    goal: |
      Conduct an in-depth analysis of products and competitors to guide marketing strategies with actionable insights.
    verbose: true
    allow_delegation: false
    tools:
      - "ScrapeWebsiteTool"
      - "InternetSearchTool"
    tasks:
      product_analysis:
        description: |
          Analyze product and competitor landscape for key trends, strengths, and areas of opportunity in the market.
        expected_output: "Detailed report with insights on competitors, market trends, and potential strategy alignments."
  chief_marketing_strategist:
    role: "Chief Marketing Strategist"
    backstory: |
      You are the Chief Marketing Strategist at a leading digital marketing agency, known for crafting bespoke strategies that drive success.
    goal: |
      Formulate comprehensive marketing strategies based on product analysis and market insights.
    verbose: true
    allow_delegation: false
    tools:
      - "ScrapeWebsiteTool"
      - "InternetSearchTool"
      - "InstagramSearchTool"
    tasks:
      strategy_formulation:
        description: |
          Synthesize insights from product analysis to develop impactful marketing strategies.
        expected_output: "Strategic recommendations for marketing based on product insights and competitor landscape."
  creative_content_creator:
    role: "Creative Content Creator"
    backstory: |
      As a Creative Content Creator at a top-tier digital marketing agency, you excel in crafting narratives that resonate with audiences on social media. Your expertise lies in turning strategies into engaging stories and visual content.
    goal: |
      Create compelling social media content for campaigns, focusing on high-impact Instagram ad copies.
    verbose: true
    allow_delegation: true
    tools:
      - "ScrapeWebsiteTool"
      - "InternetSearchTool"
      - "InstagramSearchTool"
    tasks:
      content_creation:
        description: |
          Develop social media content that aligns with the marketing strategy and captures the target audience’s attention.
        expected_output: "Engaging social media content, optimized for platforms like Instagram, to enhance campaign reach."
  senior_photographer:
    role: "Senior Photographer"
    backstory: |
      As a Senior Photographer at a leading digital marketing agency, you are an expert in capturing photographs that inspire and engage. You are currently working on an important campaign to take standout visuals.
    goal: |
      Capture high-quality photographs for Instagram ads that convey emotion and resonate with the audience.
    verbose: true
    allow_delegation: false
    tools:
      - "InstagramSearchTool"
    tasks:
      photography:
        description: |
          Produce visually compelling photographs that align with the campaign’s objectives and enhance the ad’s impact.
        expected_output: "High-quality photographs suitable for Instagram ads that capture audience attention and engagement."
  chief_creative_director:
    role: "Chief Creative Director"
    backstory: |
      You're the Chief Content Officer of a leading digital marketing agency specializing in product branding. You ensure that your team delivers top-tier content aligned with client goals.
    goal: |
      Oversee and refine content produced by the team, ensuring alignment with the brand’s objectives and quality standards.
    verbose: true
    allow_delegation: true
    tools:
      - "ScrapeWebsiteTool"
      - "InternetSearchTool"
      - "InstagramSearchTool"
    tasks:
      review_content:
        description: |
          Evaluate content produced by team members, provide feedback, and approve or request refinements as needed.
        expected_output: "Reviewed and refined content aligned with campaign and client goals."
dependencies: []
"""

## Main

In [ ]:
import os
from praisonai import PraisonAI
from google.colab import userdata

# Create a PraisonAI instance with the agent_yaml content
praisonai = PraisonAI(agent_yaml=agent_yaml, tools=[InternetSearchTool, ScrapeWebsiteTool]) # Add InstagramSearchTool once scripted

# Add OPENAI_API_KEY Secrets to Google Colab on the Left Hand Side 🔑 or Enter Manually Below
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') or "ENTER OPENAI_API_KEY HERE"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

# Run PraisonAI
result = praisonai.run()

# Print the result
print(result) # 2/10



 [2024-11-01 14:18:36][DEBUG]: == Working Agent: Lead Market Analyst
 [2024-11-01 14:18:36][INFO]: == Starting Task: Analyze product and competitor landscape for key trends, strengths, and areas of opportunity in the market.



> Entering new CrewAgentExecutor chain...
To begin the analysis of the product and competitor landscape, I need to gather information on key competitors and market trends in the digital marketing industry. I will start by searching for the latest trends and competitor information.

Action: InternetSearchTool
Action Input: {"query": "latest trends in digital marketing 2023"}

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

[{'title': 'The Top 14 Digital Marketing Trends of 2023, According to Experts', 'href': 'https://www.wordstream.com/blog/ws/2023/01/20/digital-marketing-trends', 'body': '2023 digital marketing trends & predictions, according to experts. To get a well-rounded perspective on how the 2023 digital marketing trends will impact consumers, marketers, and brands, we came up with some key questions and interviewed marketers, advertisers, and business owners across verticals. ... "AI offers new ideas and angles that we ...'}, {'title': '9 Cutting-Edge Digital Marketing Trends for 2023 - Marketing', 'href': 'https://elements.envato.com/learn/digital-marketing-trends-2023', 'body': 'We predict 2023 to be a year of embracing new technology, revisiting traditional marketing methods, and doing things differently. So what digital marketing trends will shape the coming year? From the "Gen Z effect" to AI domination, get ahead with these top digital marketing trends for 2023.'}, {'title': '2023 Digi

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

The Top 14 Digital Marketing Trends of 2023, According to Experts
Blog
Free Tools
	Google AdsPerformance Grader
		Find out if you're making costly mistakes—and how to fix them.
Get your grade
	Facebook AdsPerformance Grader
		Get ready to improve your reach, results, and ROI—fast.
Get your grade
FreeKeyword Tool
			Discover the best keywords for your PPC and SEO goals.
Find your keywords
FreeWebsite Grader
		See how your website stacks up against the competition.
Get your grade
See More Free Tools
ResourcesGuides135 Best Words & Phrases for Marketing with Emotion 
PPC 101: Complete Guide to Google Ads
25 Ways to Increase Traffic to Your Website
Recommended ReadsGoogle Ads Benchmarks for Your Industry
Facebook Ads Benchmarks for Every Industry
15 Insanely Effective Sales Promotion Examples
PPC 101What is PPC?
The Last Guide to Account Structure You’ll Ever Need
Intro to Social Media Marketing
Recent Articles
 42 Veterans Day Greetings & Messages for Any Audience (+6 Quick Tips!) Want

...

 

[{'title': 'Top 10 AI Influencers Making Waves on Instagram - Influencer Marketing Hub', 'href': 'https://influencermarketinghub.com/ai-influencers-instagram/', 'body': "Top 10 AI Influencers on Instagram. Now that we have gone over what AI influencers are, how they are created, and why they can be a better option against regular influencers for influencer marketing, let's head to the list of the top 10 AI influencers on Instagram. 1. Lu do Magalu. 7.2M followers."}, {'title': 'How the Best Brand-Influencer Partnerships Reach Gen Z', 'href': 'https://hbr.org/2023/06/how-the-best-brand-influencer-partnerships-reach-gen-z', 'body': "How the Best Brand-Influencer Partnerships Reach Gen Z. Summary. Authenticity is among Gen Z's most important values. They feel empowered to ask and answer their own questions in a variety of ..."}, {'title': 'Gen Z has a surprising opinion about AI influencers on social media ...', 'href': 'https://nypost.com/2024/05/08/tech/gen-z-has-a-surprising-opinion

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

[{'title': 'Gen Z has a surprising opinion about AI influencers on social media ...', 'href': 'https://nypost.com/2024/05/08/tech/gen-z-has-a-surprising-opinion-about-ai-influencers-on-social-media-study-finds/', 'body': "Only 35% of Gen Z consumers considered authenticity in influencers to be a top trait they care about, according to Sprout Social's 2024 Influencer Marketing Report. Social media influencers were ..."}, {'title': 'How the Best Brand-Influencer Partnerships Reach Gen Z', 'href': 'https://hbr.org/2023/06/how-the-best-brand-influencer-partnerships-reach-gen-z', 'body': "How the Best Brand-Influencer Partnerships Reach Gen Z. Summary. Authenticity is among Gen Z's most important values. They feel empowered to ask and answer their own questions in a variety of ..."}, {'title': 'The Ultimate List Of The Top 30 Gen Z Influencers For Epic ...', 'href': 'https://billo.app/gen-z-influencers/', 'body': 'Fortunately, this post has answers. We explore the top-rated Gen Z influen

### Task Output ###
Unfortunately, despite my efforts to obtain the photographs captured for the Instagram campaign targeting Gen Z, I 
was unable to access them using the available tools or through direct communication with the team members. 

To proceed, I recommend directly contacting the team responsible for the campaign within the agency, as they should
have access to the images and can provide them for evaluation. Once you have the photographs, you can review and 
refine the content to ensure it aligns with the campaign's themes of authenticity, diversity, and personalization, 
as outlined in the campaign goals. Focus on capturing real individuals expressing their values for "Values 
Unleashed," using vibrant and playful visuals for "Fun with AI," showcasing genuine behind-the-scenes moments for 
"Behind the Brand," and highlighting relatable, real-life uses of products in influencer collaborations. These 
approaches are designed to resonate with Gen Z's preference for authenticity, innovation, and community connection.

None
